In [1]:
import pandas as pd
from tqdm import tqdm
import re
import spacy
from tqdm import tqdm
from unidecode import unidecode
from dateutil import parser
from datetime import datetime
import collections
from sklearn.feature_extraction.text import TfidfVectorizer

nlp = spacy.load('en')

In [2]:
def name_disambiguation(name):
    if (name == "Khaleda" or name == "Zia" or name == "Begum Khaleda Zia"):
        return "Khaleda Zia"
    if (name == "Hasina" or name == "Sheikh"):
        return "Sheikh Hasina"
    if (name == "Fakhrul"):
        return "Mirza Fakhrul Islam Alamgir"
    if (name == "Muhith" or name == "AMA Muhith" or name == "MA Muhith"):
        return "Abul Maal Abdul Muhith"
    if (name == "Nizami" or name == "Motiur Rahman"):
        return "Motiur Rahman Nizami"
    if (name == "Modi"):
        return "Narendra Modi"
    if (name == "Bangabandhu" or name == "Sheikh Mujib" or name == "Sheikh Mujib" or name == "Bangabandhu Sheikh Mujibur" or name == "Sheikh Mujibur Rahman"):
        return "Bangabandhu Sheikh Mujibur Rahman"
    if (name == "Tarique"):
        return "Tarique Rahman"
    if (name == "Avijit"):
        return "Avijit Roy"
    if (name == "Mozena"):
        return "Dan Mozena"
    if (name == "Yunus" or name == "Mohammad Yunus"):
        return "Muhammad Yunus"

### Date limit: 2013-07-08 to 2016-06-14 on DT, Daily Star and Daily Sun

In [3]:
dt = pd.read_json('Data/DT/bd_news_dt.json')
dstar = pd.read_json('Data/DS/news_db.json', lines=True)
dsun = pd.read_pickle('Data/Daily Sun/DailySun_ent_1.pkl')

In [4]:
def conv_to_datetime(date_dict):
    new_datetime = parser.parse(list(date_dict.items())[0][1], ignoretz=True)
    new_datetime = new_datetime.replace(hour=0, minute=0, second=0, microsecond=0)
    return new_datetime

In [5]:
dt['news_publish_date'] = dt['news_publish_date'].apply(conv_to_datetime)

In [111]:
dt_tags = list(dt['news_original_tags'])
dt_tags = [item for sublist in dt_tags for item in sublist]
print(set(dt_tags))

{'environment', 'foreign affairs', 'development', 'education', 'bangladesh', 'crime', 'politics', 'labour', 'law and rights', 'science', 'safety', 'agriculture'}


In [112]:
dsun_tags = set(list(dsun['section']))
print(dsun_tags)

{'/metropolis', '/world-print', '/editorial', '/my-districts', '/winner', '/front-page', '/culturetainment', '/back-page', '/business-print', '/our-faith', '/asia-print', '/news-link'}


In [113]:
dstar_tags = set(list(dstar['section']))
print(dstar_tags)

{'Editorial', 'Country', 'National', 'Metropolitan', 'Wide Angle', 'Health', 'Next Step', 'Sports', 'Front Page', 'City', 'The Star', 'Business', 'Lifestyle', 'Back Page', 'Letters', 'In Focus', 'Literature', 'Shout', 'Star Chittagong', 'Showbiz', 'Opinion', 'Bytes', 'Law & Our Rights', 'Star City', 'Shift', 'Strategic Issues', 'World', 'Star Weekend', 'Book Reviews', 'Star People', 'Arts & Entertainment'}


In [60]:
dstar.columns

Index(['_id', 'about_reporter', 'bottom_tagline', 'breadcrumb', 'category',
       'content', 'date_crawled', 'date_published', 'generated_keywords',
       'generated_summary', 'id', 'image_captions', 'images', 'ml_tags',
       'ner_location', 'ner_money', 'ner_organization', 'ner_percent',
       'ner_person', 'ner_time', 'ner_unique_location', 'ner_unique_money',
       'ner_unique_organization', 'ner_unique_percent', 'ner_unique_person',
       'ner_unique_time', 'news_url', 'newspaper', 'reporter', 'section',
       'sentiment', 'shoulder', 'title', 'top_tagline'],
      dtype='object')

In [86]:
dsun.head()

,_id,date_published,image,news_content,news_id,newspaper,reporter,section,url,location_entities,organization_entities,person_entities
0,{'$oid': '5af44bf218ce4223c32284b0'},2012-06-01,"{'src': None, 'caption': None}",ATN BANGLA 19:00 22:00 23:00 Channel i 7:00 ...,32538,dailysun,None,/culturetainment,http://www.daily-sun.com/arcprint/details/3253...,[],[],[]
1,{'$oid': '5af44bf318ce4223c32284b1'},2012-06-01,{'src': 'http://www.daily-sun.com/assets/news_...,Channel i 18:00 City Bank N A Financial Quiz ...,32535,dailysun,None,/culturetainment,http://www.daily-sun.com/arcprint/details/3253...,"[Sarabhai, Catalunya, Uposhonghar, The Dark En...","[Stuart Little, Golpo Sholpo Gaan, Fan Club To...","[16:30 Drama Serial, Ghatak Baki Bhai , 19:50..."
2,{'$oid': '5af44bf318ce4223c32284b2'},2012-06-01,"{'src': None, 'caption': None}",Music lovers in Dhaka will be entertained with...,32533,dailysun,None,/culturetainment,http://www.daily-sun.com/arcprint/details/3253...,"[Hyderabad, South Asia , India, South Asia, P...","[Dhaka, the Pakistan High Commission]","[Sabri Qawals, Fateh Ali Khan, Qawwali]"
3,{'$oid': '5af44bf318ce4223c32284b3'},2012-06-01,{'src': 'http://www.daily-sun.com/assets/news_...,Group Art Exhibition Title- Freefall By- 5 y...,32536,dailysun,None,/culturetainment,http://www.daily-sun.com/arcprint/details/3253...,[Bangladesh],"[8:00pm Solo Art Exhibition, 3:00pm to 9:00...","[Gulshan-1, de Dhaka, Uttara , Dhanmondi, Ben..."
4,{'$oid': '5af44bf418ce4223c32284b4'},2012-06-01,{'src': 'http://www.daily-sun.com/assets/news_...,With a number of rich feathers in his cap inc...,32539,dailysun,None,/culturetainment,http://www.daily-sun.com/arcprint/details/3253...,"[Crafts, India, Kishoreganj, New York, Uttara,...","[GalleriKaya, Ekushey Padak , Hamiduzzaman, ...","[Hamid, Hamiduzzaman Khan, Hamiduzzaman, Ragha..."


In [67]:
dstar.head()

,_id,about_reporter,bottom_tagline,breadcrumb,category,content,date_crawled,date_published,generated_keywords,generated_summary,...,ner_unique_person,ner_unique_time,news_url,newspaper,reporter,section,sentiment,shoulder,title,top_tagline
0,{'$oid': '58afca955388c123849ed1be'},NaN,None,[],NaN,People's hopes for the best and confidence in ...,{'$date': '2017-02-24T00:54:28.809Z'},2008-01-01,"[parliamentary, fiscal, great, unfulfilled, ca...",This reminded the people of past military regi...,...,"[Quadri, Hasina, Bangla Bhai, Ershad, Ayub Qua...",[],http://www.thedailystar.net/news-detail-17307,The Daily Star,None,Front Page,NaN,NaN,Great expectations unfulfilled,None
1,{'$oid': '58afca985388c123849ed1bf'},NaN,None,[],NaN,"(L-R) Nasir Uddin, Abbas AliArrested Abbas Ali...",{'$date': '2017-02-24T00:54:31.997Z'},2008-01-01,"[stones, rab, nasir, museum, took, abbas, offi...",Abbas and Nasir said they stole the statuettes...,...,"[Mohammad Ali, Hamid, Nikhil Chandra Das, Abdu...",[],http://www.thedailystar.net/news-detail-17312,The Daily Star,None,Front Page,NaN,NaN,None,None
2,{'$oid': '58afca9a5388c123849ed1c0'},NaN,None,[],NaN,Daniel Vettori and Brendon McCullum put on rec...,{'$date': '2017-02-24T00:54:34.466Z'},2008-01-01,"[zealand, seven, balls, bangladesh, odi, vetto...",The match was over before the scheduled first ...,...,"[Aftab Ahmed, Zaheer, Jacob Oram, Vettori, Far...",[],http://www.thedailystar.net/news-detail-17353,The Daily Star,None,Front Page,NaN,NaN,Kiwis maul Tigers,None
3,{'$oid': '58afca9c5388c123849ed1c1'},NaN,Says BB governor,[],NaN,Bangladesh Bank (BB) Governor Salehuddin Ahmed...,{'$date': '2017-02-24T00:54:36.829Z'},2008-01-01,"[wont, outgoing, essentials, soon, governor, i...",Bangladesh Bank in the new year will focus on ...,...,"[Salehuddin Ahmed, Cyclone Sidr]",[],http://www.thedailystar.net/news-detail-17325,The Daily Star,None,Front Page,NaN,NaN,Essentials prices won't go down soon,None
4,{'$oid': '58afca9f5388c123849ed1c2'},NaN,None,[],NaN,Justice KM Sobhan died of cardiac arrest at Bi...,{'$date': '2017-02-24T00:54:39.231Z'},2008-01-01,"[passes, sons, sobhan, justice, away, shock, k...","Family sources said Justice Sobhan, also the p...",...,"[KM Sobhan, Ashraful Islam, Sobhan, Zillur Rah...",[],http://www.thedailystar.net/news-detail-17340,The Daily Star,None,Front Page,NaN,NaN,KM Sobhan passes away,None


In [6]:
dstar['date_published'] = dstar['date_published'].apply(conv_to_datetime)

In [7]:
dsun['date_published'] = pd.to_datetime(dsun['date_published'])

In [8]:
start_date = datetime(2013, 7, 8)
end_date = datetime(2016, 6, 14)

In [9]:
dt_new = dt.loc[(dt['news_publish_date']>=start_date) & (dt['news_publish_date']<=end_date)]

In [10]:
dstar_new = dstar.loc[(dstar['date_published']>=start_date) & (dstar['date_published']<=end_date)]

In [11]:
dsun_new = dsun.loc[(dsun['date_published']>=start_date) & (dsun['date_published']<=end_date)]

In [12]:
print("dt: {}, dstar: {}, dsun{}".format(dt_new.shape, dstar.shape, dt.shape))

dt: (46611, 17), dstar: (165236, 34), dsun(49055, 17)


In [13]:
dt_new = dt_new.reset_index(drop=True)
dstar_new = dstar_new.reset_index(drop=True)
dsun_new = dsun_new.reset_index(drop=True)

## Count

In [14]:
dsun_counts = dsun_new.groupby('date_published').sum().agg({
    'location_entities': collections.Counter, 
    'organization_entities': collections.Counter,
    'person_entities': collections.Counter
})

In [15]:
# Sorts Counter object

dsun_counts['location_entities'] = dsun_counts['location_entities'].apply(lambda x: x.most_common())
dsun_counts['organization_entities'] = dsun_counts['organization_entities'].apply(lambda x: x.most_common())
dsun_counts['person_entities'] = dsun_counts['person_entities'].apply(lambda x: x.most_common())

In [16]:
# Daily Star and DT doesn't work the same way for some absurd reason.
# Trying on a small dataset

# df = pd.read_json('Data/DS/test.json', lines=True)
# df['date_published'] = df['date_published'].apply(conv_to_datetime)
# df_counts = df.groupby('date_published').sum().agg({
#     'ner_unique_location': collections.Counter, 
#     'ner_unique_organization': collections.Counter,
#     'ner_unique_person': collections.Counter
# })

In [17]:
dstar_counts = pd.DataFrame()

In [18]:
dstar_counts['location_entities'] = dstar_new.groupby('date_published')['ner_unique_location'].sum().apply(collections.Counter, 1)
dstar_counts['organization_entities'] = dstar_new.groupby('date_published')['ner_unique_organization'].sum().apply(collections.Counter, 1)
dstar_counts['person_entities'] = dstar_new.groupby('date_published')['ner_unique_person'].sum().apply(collections.Counter, 1)

In [19]:
# Sorts Counter object


dstar_counts['location_entities'] = dstar_counts['location_entities'].apply(lambda x: x.most_common())
dstar_counts['organization_entities'] = dstar_counts['organization_entities'].apply(lambda x: x.most_common())
dstar_counts['person_entities'] = dstar_counts['person_entities'].apply(lambda x: x.most_common())

In [20]:
# Splits the news_ner_tags column, which contains dictionaries, into different columns based on dictionary keys
dt_new_modified = pd.concat([dt_new.drop(['news_ner_tags'], axis=1), dt_new['news_ner_tags'].apply(pd.Series)], axis=1)

In [21]:
dt_counts = pd.DataFrame()

In [22]:
dt_counts['location_entities'] = dt_new_modified.groupby('news_publish_date')['locations_unique'].sum().apply(collections.Counter, 1)
dt_counts['organization_entities'] = dt_new_modified.groupby('news_publish_date')['organizations_unique'].sum().apply(collections.Counter, 1)
dt_counts['person_entities'] = dt_new_modified.groupby('news_publish_date')['persons_unique'].sum().apply(collections.Counter, 1)

In [23]:
# Sorts Counter object

dt_counts['location_entities'] = dt_counts['location_entities'].apply(lambda x: x.most_common())
dt_counts['organization_entities'] = dt_counts['organization_entities'].apply(lambda x: x.most_common())
dt_counts['person_entities'] = dt_counts['person_entities'].apply(lambda x: x.most_common())

In [31]:
dt_counts

,location_entities,organization_entities,person_entities
news_publish_date,,,
2013-07-08,"[(Bangladesh, 5), (Gazipur, 4), (Dhaka, 4), (G...","[(Awami League, 6), (BNP, 4), (MA Mannan, 3), ...","[(Mannan, 2), (Ershad, 2), (Azmat Ullah Khan, ..."
2013-07-09,"[(Dhaka, 12), (Bangladesh, 10), (Gazipur, 3), ...","[(Awami League, 5), (BNP, 4), (Dhaka Tribune, ...","[(Ershad, 3), (Hannan, 2), (Mahmudur Rahman, 2..."
2013-07-10,"[(Dhaka, 7), (Bangladesh, 6), (Gazipur, 3), (N...","[(Awami League, 5), (BNP, 3), (Dhaka Medical C...","[(Sheikh Hasina, 3), (Kalam, 2), (Rab, 2), (Ha..."
2013-07-11,"[(Dhaka, 16), (Bangladesh, 10), (Chittagong, 5...","[(Dhaka Tribune, 10), (Awami League, 5), (TIB,...","[(Azad, 3), (Sheikh Hasina, 2), (Mannan, 2), (..."
2013-07-12,"[(Dhaka, 9), (Bangladesh, 6), (Gazipur, 2), (R...","[(BNP, 4), (Awami League, 4), (Dhaka Universit...","[(Abul Kalam, 2), (Hasina, 2), (Sheikh Hasina,..."
2013-07-13,"[(Dhaka, 15), (Bangladesh, 11), (Gazipur, 3), ...","[(BNP, 6), (Awami League, 5), (BGB, 2), (Awami...","[(Sheikh Hasina, 4), (Nuh-ul-Alam Lenin, 2), (..."
2013-07-14,"[(Dhaka, 5), (Bangladesh, 5), (Chittagong, 4),...","[(BNP, 3), (PSC, 3), (Securities and Exchange ...","[(Sheikh Hasina, 2), (Akash Malik, 2), (Shafi,..."
2013-07-15,"[(Dhaka, 11), (Bangladesh, 10), (Pakistan, 4),...","[(Jamaat, 9), (BNP, 6), (ICT, 6), (Internation...","[(Ghulam Azam, 13), (Ganajagaran Mancha, 3), (..."
2013-07-16,"[(Bangladesh, 9), (Dhaka, 8), (Pakistan, 5), (...","[(Jamaat, 12), (ICT, 5), (Awami League, 4), (I...","[(Ghulam Azam, 13), (Fazle Kabir, 3), (Azam, 3..."


## TF-IDF

In [24]:
tfidf = TfidfVectorizer()

In [25]:
dsun_tfidf = pd.DataFrame()
tqdm.pandas()

In [26]:
# dsun_new['location_entities'] =  dsun_new['location_entities'].progress_apply(lambda x: ' '.join(str(e) for e in x))

In [27]:
dsun_new.to_pickle('dsun_new.pkl')

In [28]:
# dsun_tfidf['location_entities'] = dsun_new.groupby('date_published').progress_apply(tfidf.fit_transform(dsun_new['location_entities']))

In [29]:
df = pd.DataFrame({'Date': ['2018-09-01','2018-09-01 ','2018-09-02'],
               'documents': [['Dhaka', 'Chittagong', 'Sylhet'],['Dhaka', 'India'],['Dhaka']]})


df['documents'] = df['documents'].apply(lambda x: ' '.join(str(e) for e in x))


In [30]:
v = TfidfVectorizer()

df['documents'] = df.groupby('Date').apply(v.fit_transform(df['documents']))
# x = v.fit_transform(df['documents'])
# df1 = pd.DataFrame(x.toarray(), columns=v.get_feature_names())
# res = pd.concat([df, df1], axis=1)

TypeError: unhashable type: 'csr_matrix'

In [ ]:
df.head()